In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os, sys, traceback

import numpy as np
from openai import OpenAI

oai_key="<YUOR_OPENAI_API_KEY>"

In [ ]:
def bs_mod(page,element_type='div',class_type="description__text description__text--rich",types=['p','ul','ol']):
  '''
  THIS FUNCTION SCRAPES LINKEDIN JOB SPEC PAGE
  INPUTS:
  - page - raw html text from request module
  - element_type - type of html element that contains the job spec
  - class_type - class of html element that contains the job spec
  - types - html subelement types from the job spec html element to process

  OUTPUTS
  - out_f - job spec in string format
  - out - job spec + additional info in pandas.DataFrame format
  '''
  out_i=[]
  out_t=[]
  out_type=[]

  soup0 = BeautifulSoup(page.content, "html.parser")
  soups=soup0.find_all(element_type, class_=class_type)

  txt = page.text

  for t in types:
    for soup in soups:
      e = soup.find_all(t)

      if isinstance(e,list):
        for ei in e:
          ind = txt.find(str(ei))
          if ind>-1:
            out_i.append(ind)
            out_t.append(str(ei))
            out_type.append(t)
      elif (e is not None) & (len(e)>0):
        ind = txt.find(str(e))
        if ind>-1:
          out_i.append(ind)
          out_t.append(str(e))
          out_type.append(t)

  # Sort by position
  out=pd.DataFrame({'Type':out_type,'Position':out_i,'Value':out_t})
  out=out.sort_values(by='Position').reset_index(drop=True)
  #out.to_csv('HTML parsed.csv',index=None)

  # Beautify and convert to string
  mapping={'<ul>':'','</ul>':' \n ','<li>':' - ','</li>':' \n '}

  for t in types:
    if t not in ['ul','li']:
      k='<'+t+'>'
      mapping[k]=''
      k='</'+t+'>'
      mapping[k]=' \n '

  for k in mapping.keys():
    out['Value']=out['Value'].astype(str).str.replace(k,mapping[k],regex=True)

  # If anything's left
  out['Value']=out['Value'].astype(str).str.replace('</.*>',' \n ',regex=True).str.replace('<.*>','',regex=True).str.replace('\n.{0,4}\n','\n',regex=True)

  out_f=' \n '.join(list(out['Value'].astype(str)))

  return(out_f,out)

In [ ]:
def generate_response(prompt,artefact=None,mask=None,model_engine = 'gpt-4',max_tokens=1024,secret_key=None,
                      mode='create',test=0,persona="You are a helpful assistant.",number=1,size='1024x1024'):

    # https://github.com/openai/openai-python
    # https://platform.openai.com/docs/models/overview
    # https://platform.openai.com/docs/api-reference/introduction
    # https://platform.openai.com/docs/api-reference/completions/create

    # MODELS: 'gpt-4','gpt-3.5-turbo', "text-davinci-003","text-davinci-002", "code-davinci-002", "dall.e"
    # MODE: only for image generation/editing: https://platform.openai.com/docs/guides/images/introduction - 'create','edit','variation'

    # If input is a text file
    if prompt.split('.')[-1].lower() in ['txt','dat','sql','py','r']:
        print('Reading ',prompt)
        with open(prompt) as f:
            prompt = f.read()

    # If additional data provided
    if artefact is not None:
        print('Additional artefact provided...')
        if artefact.split('.')[-1].lower() in ['txt','dat','sql','py','r']:
            print('Reading ',artefact)
            with open(artefact) as f:
                artefact = f.read()
            prompt=prompt+' '+artefact
        elif artefact.split('.')[-1].lower() in ['csv']:
            artefact = pd.read_csv(artefact)
            artefact = ', '.join(list(artefact.iloc[:,0].astype(str)))
            prompt=prompt+' '+artefact
        elif artefact.split('.')[-1].lower() in ['png','jpg','jpeg']:
            artefact=open(artefact, "rb")
        elif isinstance(artefact,str):
            prompt=prompt+' '+artefact

    if mask is not None:
      if mask.split('.')[-1].lower() in ['png','jpg','jpeg']:
        mask=open(mask, "rb")

    result = 'INVALID OPENAI KEY!!!'
    completions = None

    if secret_key is not None:

        print('>>> Querying chatGPT ',model_engine)
        #print(prompt)

        #prompt = (f"{prompt}")

        client = OpenAI(
          # defaults to os.environ.get("OPENAI_API_KEY")
          api_key=secret_key,
        )

        try:
            if np.sum(test)==0:
                if ('dall-e' in model_engine.lower())&(mode.lower()=='create'):

                    completions = client.images.generate(
                                  model=model_engine.lower(),
                                  prompt=prompt,
                                  n=number,
                                  size=size
                                )
                    result = completions['data'][0]['url']
                elif ('dall-e' in model_engine.lower())&(mode.lower()=='edit'):
                    completions = client.images.edit(
                                  image=artefact,
                                  mask=mask,
                                  prompt=prompt,
                                  n=number,
                                  size=size
                                )
                    result = completions['data'][0]['url']
                elif ('dall-e' in model_engine.lower())&(mode == 'variation'):
                    completions = client.images.create_variation(
                                image=artefact,
                                n=number,
                                size=size
                              )
                    result = completions['data'][0]['url']
                else:
                    completions = client.chat.completions.create(
                              model=model_engine.lower(),
                              messages=[{"role": "system", "content": persona},
                                        {"role": "user", "content": prompt}],
                              max_tokens=max_tokens,
                              n=number,
                              stop=None,
                              temperature=0.5
                          )
                    result = completions.choices[0].message.content
                    result = result.strip()

                print('>>> Query completed successfully :)')
                print('Token usage: ')
                print(completions.usage)
            else:
                print("API not queried - only testing...")
        except Exception as e:
            print(e)
            exc_type, exc_value, exc_traceback = sys.exc_info()
            traceback.print_exception(exc_type, exc_value, exc_traceback,limit=5, file=sys.stdout)
            result=e
    else:
        print('INVALID OPENAI KEY!!!')

    return(result,completions)

In [ ]:
def optimize_cv(jid,cv,secret_key=None,model_engine = "gpt-4",test=0,optimize=1,prompt=None):
  '''
  FUNCTION TO RATE OR OPTIMIZE CV IN LINE WITH A LINKEDIN JOB SPEC

  INPUTS
  - JID - LinkedIn job spec ID as specified in the job spec URL
  - CV - either filename of a .txt file with the CV, or CV in string format
  - SECRET_KEY - OpenAI API secret key
  - MODEL_ENGINE - OpenAI model spec (e.g. 'gpt-4')
  - TEST - 0 for full run, 1 for test run w/o querying of the LLM
  - OPTMIZE - 0 for CV rating, 1 for CV optimization, None for other LLM prompts
  - PROMPT - an laternative prompt (if required) - can use f-string utilizing internal variables (cv, out_f)
  '''

  if len(cv)<100:
    f=open(cv, "r")
    cv=f.read()
    f.close()

  URL = f"https://www.linkedin.com/jobs/view/{jid}/"
  page = requests.get(URL)

  out_f,out=bs_mod(page)

  if optimize is None:
    p=f"f'{prompt}'"
    prompt=eval(p)
  elif optimize:
    prompt=f'''
            I want you to act as an experienced recruitment consultant. Please optimize the following resume for ATS, keyword match and skills coverage:
            '{cv}'
            , against the following job specification:
            '{out_f}'
            '''
  else:
    prompt=f'''
            I want you to act as an experienced recruitment consultant. Please score the following resume on the scale 1 (worst) to 10 (best):
            '{cv}'
            , against the following job specification:
            '{out_f}'
            '''
  answer=None
  if secret_key is not None:
    try:
      answer,compl=generate_response(prompt,model_engine = model_engine,secret_key=secret_key,test=test)
    except Exception as e:
      print(e)
      exc_type, exc_value, exc_traceback = sys.exc_info()
      traceback.print_exception(exc_type, exc_value, exc_traceback,limit=5, file=sys.stdout)
  else:
    print('ERROR: No openAI API secret key provided!!!')

  # Save to txt file

  return(answer)

In [ ]:
# LINKEDIN JOB ID
#jid='3764251400'
jid='3761787132'

In [ ]:
# Example of use of BS_MOD()
URL = f"https://www.linkedin.com/jobs/view/{jid}/"
page = requests.get(URL)
out_f,out=bs_mod(page)
display(out)
print(out_f)

,Type,Position,Value
0,p,47434,Senior Analytics &amp; Performance Manager \n
1,p,47500,"Salary: £85,000 – £90,000 + benefits \n"
2,p,47561,Location: London (2days in the office) \n
3,p,47635,(Sponsorship is not offered with this role) \n
4,p,47720,Sure Exec Search are currently working with a ...
5,p,47895,This role requires a seasoned professional who...
6,p,48382,Job Responsibilities \n
7,ol,48426,- Leadership: \n Supervising and coaching a ...
8,p,49494,Essential Qualifications / Experience \n
9,ol,49556,- Experience: \n ETL Processes - 5+ years in...


Senior Analytics &amp; Performance Manager 
  
 Salary: £85,000 – £90,000 + benefits  
  
 Location: London (2days in the office)  
  
 (Sponsorship is not offered with this role) 
  
 Sure Exec Search are currently working with a London based client who is looking for a Senior Analytics &amp; Performance Manager to join their growing team. 
  
 This role requires a seasoned professional who can lead a team in using data-driven insights to enhance business operations. The emphasis on Tableau expertise, along with a mix of technical and soft skills, underscores the importance of both the technological and communicative aspects of the job. Experience in Extract, transform, and load (ETL) processes is essential and candidates will need to be able to demonstrate there use of ETL within there most recent role.  
  
 Job Responsibilities 
  
  - Leadership: 
  Supervising and coaching a team in data analytics and management to enhance the company's decision-making abilities. 
  - Vision: 
  

In [ ]:
# SAMPLE CV
cv='''
Josef (Joe) Trchalik, MSc PhD
Senior Manager, Data Analytics & Data Science
West Oxfordshire (OX18), UK ▪ +44 (0) 7891 653 745
Age: 43 years ▪ Nationality: UK & EU
email: jtrchal@yahoo.co.uk  ▪ Linkedin: www.linkedin.com/in/joseftrchalik

Summary
Experienced people leader with an extensive hands-on expertise in business analytics, data science, machine learning and process improvement. Recruited, trained and managed teams to support complex cross-functional business improvement projects. Enabled business benefits exceeding £120M in cost and revenue and 1M man-hours in efficiency. Lean Six Sigma Black Belt certified, managed process improvement initiatives and training programs across business functions and sites. Passionate about continuous learning and staying updated with latest industry trends.
Core Skill Profile
Data Analysis | Data Science | Machine Learning | People Management | Team Leadership & Training | Project Management
Process Improvement | Lean Six Sigma Blackbelt | Problem Solving
Professional Experience
Radical Simplification & Automation, Vodafone, UK	2019 – Present
Senior Manager, Data Analytics & Data Science
Leading Data Analytics & Data Science team in Radical Simplification & Automation organisation (RSA), an internal consultancy at Vodafone UK. Member of RSA's Senior Leadership Team, reporting directly to Head of RSA. Exclusively working on complex, cross-functional process improvement initiatives mandated from CEO or Director level. Responsible for delivery of data science, reporting and machine learning solutions essential for RSA’s deliverables. Recruited, trained and managed team of five data analysts & data scientists.
Managed delivery of data analyses and insights that enabled achieving RSA’s business benefits: efficiency gains of over 1M man-hours, revenue uplift exceeding £65M and cost reduction of over £60M. Benefits delivered by my team included:
•	Generated £5.2M revenue uplift by developing cloud-based Data Integrity Analysis application.
•	Gained £2.3M revenue by building data pipelines and reporting dashboards for Lead-to-Order project.
•	Accomplished £1.2M cost reduction by executing process mining analysis of customer journeys.
•	Raised £0.9M in revenue by implementing NLP models of customer Order-to-Activation journeys.
•	Achieved £1M cost reduction by designing Qliksense retail store scenario analysis tool.
•	Developed clustering and deep learning time series models for store segmentation and predictive analysis.
•	Pioneered use of chatGPT LLM API in RSA for source code generation, debugging and documentation.
Avionics UK Operations, GE Aviation Systems, UK	2019 – 2017
Analytics Transformation Leader
Led skilled team utilising analytics and machine learning to enhance operations in UK Operations organisation. Functioned as subject matter expert for applied statistics, data science, and Lean Six Sigma within UK Operations team. Collaborated as co-leader and instructor for GE LSS Green Belt and GE Digital Data Analyst training programmes.
•	Driving use of data science by facilitating global analytics steering committee and site-focused task force.
•	Successfully executed business improvement projects across several sites.
•	Pioneered creation and implementation of automation and reporting tools, resulting in productivity gain of £1M.
Central Operations, GE Aviation Systems, UK	2014 – 2017
Lean Six Sigma Black Belt
Directed process improvement initiatives and deployed Lean Six Sigma methodologies to improve business functions and operations across multiple locations. Streamlined efforts across all GE AS sites by coordinating as engineering and material cost productivity focal point. Mentored aspiring Green Belt candidates and collaborated in enhancement of Green Belt training materials based on valuable feedback from participants.
•	Generated £1.4M in cost reductions through effective leadership in non-conformance process improvements.
•	Oversaw implementation of savings tracker tool by supervising dedicated IT team in GE IQ Mexico.
•	Organised Green Belt and Lean Startup/FastWorks training sessions across different GE Aviation sites in UK & USA.
Dowty Propellers, GE Aviation Systems, UK	2011 – 2014
Engineering Project Manager (EPM)
Spearheaded planning, coordinating, and execution of Boeing Apache NG Tail Rotor project from concept to completion, ensuring adherence to project timelines, budget, and quality standards. Engaged with cross-functional teams, including engineers, technicians, and vendors, to define project scope, deliverables, and milestones.
•	Directed engineering design and development of SSC propeller system, from contract award in January 2013 through bid phase, programme launch, and NPI execution.
•	Managed $10M+ US Navy Ship-to-Shore Connector (SSC) propeller system programme as project manager.
Performance Engineer, Dowty Propellers, GE Aviation Systems, UK (2007 – 2011)
Project Engineer, Aerogistics Ltd, UK (2004 – 2005)
Academic and Professional Qualifications
Doctorate (PhD) by Research in Aerospace Engineering, University of Glasgow, UK
Aeroelastic Modelling of Gyroplane Rotors PhD research project funded by the UK CAA. Developed a standalone MATLAB aeroelastic model of gyroplane rotor using Finite Element Method.
Master of Science (Ing/MSc) in Aerospace Engineering, Brno University of Technology (VUT), Czech Republic
UK NARIC certified equivalency to British taught Master’s Degree.

Analytics & Data Science
GE Aviation Digital Data Analyst certification & instructor training | GE Aviation Digital Data Scientist certification | MOOCs: Machine Learning (Andrew Ng), Analytics Edge, Statistical Learning
Process Excellence
GE Change Acceleration Process | GE Lean Six Sigma Green Belt certification & instructor training | GE FastWorks Everyday certification & instructor training | GE Lean Six Sigma Black Belt certification | Hoshin Kanri / Strategy Deployment & GE Aviation Performance System training | GE Lean Academy | Shainin RedX Apprentice.
Business Leadership & Project Management
GE Crotonville Foundations of Leadership | GE Project Management University certification | PMI PMP certification | GE Crotonville Authentic Leadership | GE Digital Industrial Leader training series
Security Clearance
UK SC (2023)
Technical Skills
Programming: Python, SQL, R, Matlab/Octave, Fortran.
Platforms: Windows, Unix/Linux; Cloud: AWS, GCP.
Data Science & ML: Python (incl. pandas, scikit-learn, Pytorch, TensorFlow, openAI), AWS Sagemaker, RStudio, AWS EMR + pyspark, GCP BQ, Dataiku, Celonis + pycelonis, Minitab.
Databases: Oracle SQL, Teradata SQL, MySQL, postgreSQL, PQL, mongoDB + pymongo, Alation.
Dashboarding: TIBCO Spotfire (hands-on + instructor), familiar with Tableau, Qliksense and PowerBI.
Other: GitHub, MS Office, MS Project, MS Visio.
Honours & Awards
GE Aviation Systems 2013 Open Innovation Challenge finalist
GE Aviation Systems Q4 2012 Growth Value Award - SSC Integrated Project Team – Team of the Quarter
2019 GE Aviation Systems Engineering Recognition Day Winner - Cost Out
2022 Vodafone UK Finance Quarterly Team Award – Home Broadband Data Integrity Analysis
'''

In [ ]:
r1=optimize_cv(jid,cv,secret_key=oai_key,optimize=0)

>>> Querying chatGPT  gpt-4
>>> Query completed successfully :)
Token usage: 
CompletionUsage(completion_tokens=369, prompt_tokens=2041, total_tokens=2410)


In [ ]:
r2=optimize_cv(jid,cv,secret_key=oai_key,optimize=1)

>>> Querying chatGPT  gpt-4
>>> Query completed successfully :)
Token usage: 
CompletionUsage(completion_tokens=774, prompt_tokens=2035, total_tokens=2809)


In [ ]:
print(r1)

After a thorough review of Josef Trchalik's resume, I would rate it an 8.5 out of 10 against the job specification for the Senior Analytics & Performance Manager role. Here's why:

Strengths:
- Josef has a strong background in data analytics and data science, with senior management experience, which aligns well with the requirement for a seasoned professional who can lead a team in using data-driven insights.
- He has a proven track record of leading teams and managing complex projects, which aligns with the leadership and operations responsibilities of the role.
- His experience in developing and applying analytical methods and improving data systems, as well as his work with a variety of data types and programming languages (Python, SQL, R), aligns with the Analytics and Technical Skills requirements.
- Josef has an impressive academic background with a PhD in Aerospace Engineering and an MSc in the same field, which demonstrates a high level of analytical and problem-solving skills.

In [ ]:
print(r2)

Here is the optimized version of the resume:

Josef (Joe) Trchalik, MSc PhD
Senior Analytics & Performance Manager
West Oxfordshire (OX18), UK ▪ +44 (0) 7891 653 745
UK & EU Citizen
Email: jtrchal@yahoo.co.uk  ▪ Linkedin: www.linkedin.com/in/joseftrchalik

Summary
Seasoned analytics professional with extensive expertise in data science, machine learning, and business analytics. Proven track record in leading teams, executing data-driven strategies, and enhancing business operations. Demonstrated success in delivering business benefits worth over £120M in cost and revenue and 1M man-hours in efficiency. Lean Six Sigma Black Belt certified, with proficiency in process improvement and project management.

Skills
Data Analytics | Data Science | Machine Learning | Team Leadership & Training | Project Management | Process Improvement | Lean Six Sigma Black Belt | Problem Solving | ETL Processes | Tableau | SQL | Python | R

Professional Experience
Vodafone, UK (2019 – Present)
Senior Manager